In [20]:
from SimPEG import Mesh, Maps
from simpegEM1D import *
import numpy as np
%pylab inline

nearthick = np.logspace(-1, 1, 5)
deepthick = np.logspace(1, 2, 10)
hx = np.r_[nearthick, deepthick]
mesh1D = Mesh.TensorMesh([hx], [0.])
depth = -mesh1D.gridN[:-1]
LocSigZ = -mesh1D.gridCC
nlay = depth.size
topo = np.r_[0., 0., 30.]

FDsurvey = EM1DSurveyFD(
    rx_location = np.array([0., 0., 30.]),
    src_location = np.array([0., 0., 30.]),
    field_type = 'secondary',
    rx_type = 'Hz',
    src_type = 'VMD',
    depth = depth,
    topo = topo,
    frequency = np.r_[100.],
    offset = np.r_[8.]
)

sig_half = 1e-2
chi_half = 0.
Expmap = Maps.ExpMap(mesh1D)

# Conductivity
prob = EM1D(
    mesh1D, sigmaMap=Expmap, jacSwitch=False,
    chi=np.zeros(FDsurvey.n_layer)
)
if prob.ispaired:
    prob.unpair()
if FDsurvey.ispaired:
    FDsurvey.unpair()
prob.pair(FDsurvey)
m_1D = np.log(np.ones(nlay)*sig_half)
Hz = FDsurvey.dpred(m_1D)

Populating the interactive namespace from numpy and matplotlib


/Users/sgkang/anaconda2/envs/simpeg/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['pi', 'ioff']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [21]:
import numpy as np
from scipy.interpolate import InterpolatedUnivariateSpline as iuSpline
from empymod.filters import key_201_2009 as fhtfilt  # Chance this to your choice


def get_spline_values(filt, inp):
    """Return required calculation points."""

    # Number per decade from filter.factor
    nr_per_dec = 1/np.log(filt.factor)

    # Get min and max required out-values (depends on filter and inp-value)
    outmax = filt.base[-1]/inp.min()
    outmin = filt.base[0]/inp.max()

    # Number of out-values
    nout = int(np.ceil(np.log(outmax/outmin)*nr_per_dec) + 1)
    # The cubic InterpolatedUnivariateSpline needs at least 4 points
    if nout-filt.base.size < 3:
        nout = filt.base.size+3

    # Calculate output values
    out = np.exp(np.arange(np.log(outmin), np.log(outmin) + nout/nr_per_dec,
                           1/nr_per_dec))

    # Only necessary if standard spline is used. We need to calculate the new
    # input values, as spline is carried out in the input domain. Else spline
    # is carried out in output domain and the new input values are not used.
    new_inp = inp.max()*np.exp(-np.arange(nout - filt.base.size + 1) /
                               nr_per_dec)

    # Return output values
    return np.atleast_2d(out), new_inp

# 1. COMPUTE REQUIRED LAMBDAS for given hankel-filter-base

from empymod.filters import key_201_2009
fht_filter = key_201_2009()
off = FDsurvey.offset
lambd, ioff = get_spline_values(fht_filter, off)
print (lambd.size, ioff.size)

204 4


In [22]:
ioff

array([8.        , 7.42937355, 6.89944892, 6.40732291])

In [23]:
PJ0 = prob.hz_kernel_vertical_magnetic_dipole(
    lambd/FDsurvey.offset, FDsurvey.frequency, FDsurvey.n_layer,
    prob.sigma, prob.chi, FDsurvey.depth, FDsurvey.h, FDsurvey.z,
    'secondary', output_type='response'
)

In [24]:
# PJ0 = np.empty((off.size, lambd.size), dtype=complex)
# PJ1 = np.empty((off.size, lambd.size), dtype=complex)

In [25]:
# 2. CALL THE KERNEL
# Here comes your PJ0, PJ1 calculation

def rearrange_PJ(PJ, noff, nfilt):
    """Return re-arranged PJ with shape (noff, nlambd).
        Each row starts one 'lambda' higher."""
    outarr = np.concatenate((np.tile(PJ, noff).squeeze(), np.zeros(noff)))
    return outarr.reshape(noff, -1)[:, :nfilt]

PJ0 = rearrange_PJ(PJ0, ioff.size, fht_filter.base.size)
# PJ1 = rearrange_PJ(PJ1, ioff.size, fht_filter.base.size)

# 3. DLF
# EM_int = np.dot(PJ1, fht_filter.j1)/ioff + np.dot(PJ0, fht_filter.j0)
EM_int = np.dot(PJ0, fht_filter.j0)

# 4. Interpolation
real_EM = iuSpline(np.log10(ioff[::-1]), EM_int.real[::-1])
imag_EM = iuSpline(np.log10(ioff[::-1]), EM_int.imag[::-1])
fEM = real_EM(np.log10(off)) + 1j*imag_EM(np.log10(off))

# Normalize by offset

fEM /= off

In [26]:
print (Hz)
print (fEM)

[-3.27993936e-10 -1.93020685e-08]
[-2.35807444e-09-1.69832723e-08j]


In [27]:
iuSpline??